In [1]:
import pandas as pds
from rdflib import Graph, RDFS, RDF, OWL, URIRef, Literal
from hashlib import md5
from pandasql import sqldf

def pysqldf(q):
    return sqldf(q, globals())

### Load data from classificaiton paths file `GOLD_Ecosystem_Classification_Paths_10152019.xlsx`

In [4]:
pathsdf = pds.read_excel("data/GOLD_Ecosystem_Classification_Paths_10152019.xlsx")

### Clean data: 
* replace with nan with empyt string
* make all values lowercase 
* trim spaces

In [22]:
pathsdf.fillna("", inplace=True)
pathsdf = pathsdf.applymap(lambda x: "" if "(null)" == x else x)
pathsdf = pathsdf.applymap(lambda x: x.lower().strip())    
pathsdf.head()

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM
0,engineered,bioreactor,aerobic,unclassified,unclassified
1,engineered,bioreactor,anaerobic,unclassified,unclassified
2,engineered,bioreactor,continuous culture,marine intertidal flat sediment inoculum,wadden sea-germany
3,engineered,bioreactor,continuous culture,marine sediment inoculum,wadden sea-germany
4,engineered,bioreactor,continuous culture,unclassified,unclassified


## Build dataframe of unique label paths and their checksums
#### For example: 'enviromental > aquatic > freshwater > sediment'

In [23]:
## helper functions for creating label path, hash, and iri
def make_label_path(row, include_missing=False):
    path_list = list(row)
    temp = [e for e in path_list if e != ""] # remove all empty strings
    if len(temp) > 0:
        if include_missing:
            ## if the value in the list an empty string (e.g., ['host-associated', 'plants', 'endosphere', ''])
            ## this will put an " > " it (e.g., host-associated > plants > endosphere >)
            return " > ".join(path_list)
        else:
            ## this only retuns a path devoid of empty string
            ## e.g, ['host-associated', 'plants', 'endosphere', ''] returns host-associated > plants > endosphere
            return " > ".join(temp)
    else:
        return ""

def make_parent_label_path(label_path):
    path_list = label_path.split(" > ")
    if len(path_list) > 1:
        temp = path_list[0:-1]
        return " > ".join(temp)
    else:
        return ""
    
def make_hash(val):
    if len(val) > 0:
        hash = md5(val.encode('utf-8'))
        return str(hash.hexdigest())
    else:
        return ""

def make_iri(val, prefix="http://purl.obolibrary.org/obo/GOLD_"):
    if None != val and len(val) > 0:
        hash = make_hash(val)
        return f"{prefix}{hash}"
    else:
        return ""

In [24]:
ontdf = pds.DataFrame(columns=['label_path', 'parent_label_path', 'iri', 'parent_iri'])
cols = pathsdf.columns

for i in range(len(cols)):
    df = pathsdf[cols[0:i + 1]].fillna("").drop_duplicates()
    df['label_path'] = df.apply(lambda row: make_label_path(row), axis=1)
    df['parent_label_path'] = df.apply(lambda row: make_parent_label_path(row['label_path']), axis=1)
    df['iri'] = df.apply(lambda row: make_iri(row['label_path']), axis=1)
    df['parent_iri'] = df.apply(lambda row: make_iri(row['parent_label_path']), axis=1)
    ontdf = ontdf.append(df[['label_path', 'parent_label_path', 'iri', 'parent_iri']]).drop_duplicates()

In [25]:
## visually examine output
# pds.set_option('max_rows', None)
# pds.set_option('display.max_colwidth', 1000)
# print(ontdf[['label_path', 'parent_label_path']])
# print(len(ontdf))

## Use rdflib to build ontology from dataframe

In [26]:
g = Graph()

for (ix, label_path, parent_label_path, iri, parent_iri) in ontdf.itertuples(): # ontdf.head(100).itertuples():
    if len(iri) > 0:
        g.add((URIRef(iri), RDF.type, OWL.Class))
        g.add((URIRef(iri), RDFS.label, Literal(label_path)))
    
    if len(parent_iri) > 0:
        g.add((URIRef(iri), RDFS.subClassOf, URIRef(parent_iri)))

In [27]:
g.serialize(destination='output/gold-classification-paths-translation.owl', format='xml')